# 3. GroupTesting

In [7]:
import cvxpy as cp
import numpy as np

**$\bullet$ group_testing.py :**

In [3]:
# Construct the problem
N = 1000 # Population size
S = 10 # Number of infected individuals
M = 100 # Number of tests
K = 10 # Number of splits of each sample
# Define x0
ind0 = np.random.choice(N,S,0) # index subset 
x0 = np.zeros(N) 
x0[ind0] = np.random.rand(S)

# Define A
A = np.zeros((M,N))
for i in np.arange(N):
    ind = np.random.choice(M,K,replace=False)
    A[ind,i] = 1

y = A @ x0

## (a)

In [4]:
x = cp.Variable(N, nonneg=True)
objective = cp.Minimize(cp.norm(x, 1))
constraints = [A @ x == y]
problem = cp.Problem(objective, constraints)
problem.solve()

x_estimated = x.value
estimated_indices = np.argsort(x_estimated)[-S:]

print("True infected indices:", [int(index) for index in sorted(ind0)])
print("Estimated infected indices:", [int(index) for index in sorted(estimated_indices)])

correct_identifications = set(ind0).intersection(set(estimated_indices))
print(f"Correct identifications: {len(correct_identifications)}/10")

True infected indices: [70, 319, 420, 432, 507, 562, 736, 867, 915, 965]
Estimated infected indices: [70, 319, 420, 432, 507, 562, 736, 867, 915, 965]
Correct identifications: 10/10


## (b)

As $K$ decreases, the number of tests each person contributes to reduces, which can make it harder for the optimization problem to recover the correct sparse vector $x$. We expect that below a certain threshold of $K$, the recovery accuracy will drop significantly.

In [5]:
def test_group_testing(K):
    A = np.zeros((M, N))
    for i in range(N):
        ind = np.random.choice(M, K, replace=False)
        A[ind, i] = 1

    y = A @ x0

    x = cp.Variable(N, nonneg=True)
    objective = cp.Minimize(cp.norm(x, 1))
    constraints = [A @ x == y]
    problem = cp.Problem(objective, constraints)
    problem.solve()

    x_estimated = x.value
    estimated_indices = np.argsort(x_estimated)[-S:]

    correct_identifications = len(set(ind0).intersection(set(estimated_indices)))
    return correct_identifications == S

K_values = list(range(1, 21))
results = {}
for K in K_values:
    success = test_group_testing(K)
    results[K] = success
    print(f"K = {K}: {'Success' if success else 'Failure'}")

print("\nMinimum K where recovery Success:", min([k for k, v in results.items() if v]))


K = 1: Failure
K = 2: Failure
K = 3: Success
K = 4: Success
K = 5: Success
K = 6: Success
K = 7: Success
K = 8: Success
K = 9: Success
K = 10: Success
K = 11: Success
K = 12: Success
K = 13: Success
K = 14: Success
K = 15: Success
K = 16: Success
K = 17: Success
K = 18: Success
K = 19: Success
K = 20: Success

Minimum K where recovery Success: 3


## (c)

As the number of infected individuals increases, the vector $x$ becomes less sparse. This increase in density makes it more challenging for the group testing approach to recover the infected individuals accurately, especially when $M$ is fixed and smaller than $N$. To achieve accurate recovery at higher prevalence levels, we may need to increase $K$ to ensure sufficient overlap between samples for each test.  
For very low prevalence levels we expect smaller $K$ values (around 5-10) to work well. As prevalence increases, we expect higher $K$ values to be necessary, likely around 15 or 20 for prevalence levels close to the threshold where the method fails.

In [6]:
prevalence_levels = [0.01, 0.02, 0.03, 0.05, 0.1, 0.15, 0.2]
K_values = range(1, 21)

def test_group_testing(prevalence, K):
    S = int(prevalence * N)
    x0 = np.zeros(N)
    ind0 = np.random.choice(N, S, 0)
    x0[ind0] = np.random.uniform(1, 10, S)

    A = np.zeros((M, N))
    for i in range(N):
        ind = np.random.choice(M, K, replace=False)
        A[ind, i] = 1

    y = A @ x0

    x = cp.Variable(N, nonneg=True)
    objective = cp.Minimize(cp.norm(x, 1))
    constraints = [A @ x == y]
    problem = cp.Problem(objective, constraints)
    problem.solve()

    x_estimated = x.value
    estimated_indices = np.argsort(x_estimated)[-S:]

    correct_identifications = len(set(ind0).intersection(set(estimated_indices)))
    return correct_identifications == S

results = {}
for prevalence in prevalence_levels:
    print(f"\nTesting for prevalence: {prevalence * 100}%")
    success_for_prevalence = False
    for K in K_values:
        success = test_group_testing(prevalence, K)
        if success:
            print(f"Success with K = {K}")
            results[prevalence] = K
            success_for_prevalence = True
            break
    if not success_for_prevalence:
        print("Failed to recover with any tested K")

print("\nOptimal K values for each prevalence level:")
for prevalence, optimal_K in results.items():
    print(f"Prevalence {prevalence * 100}%: Optimal K = {optimal_K}")


Testing for prevalence: 1.0%
Success with K = 3

Testing for prevalence: 2.0%
Success with K = 5

Testing for prevalence: 3.0%
Success with K = 15

Testing for prevalence: 5.0%
Failed to recover with any tested K

Testing for prevalence: 10.0%
Failed to recover with any tested K

Testing for prevalence: 15.0%
Failed to recover with any tested K

Testing for prevalence: 20.0%
Failed to recover with any tested K

Optimal K values for each prevalence level:
Prevalence 1.0%: Optimal K = 3
Prevalence 2.0%: Optimal K = 5
Prevalence 3.0%: Optimal K = 15
